In [1]:
from operator import itemgetter
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
from langchain.schema import StrOutputParser
from langchain.schema import BaseOutputParser
import json
import pandas as pd

from dotenv import load_dotenv
load_dotenv()

model = ChatOpenAI(
    # model='gpt-3.5-turbo-16k-0613',
    model='gpt-3.5-turbo-1106',
    temperature = 0.1
)

In [2]:
prompt1 = ChatPromptTemplate.from_template("""
from the given social media posts from people in the given province. please analize base on un sdg goals. return output only in json format

### province
ปทุมธานี

### social media posts
ปทุมธานีรถติดมาก
อุบัติเหตุหน้ามธ.รถติดยาว
มธ.เดินทางไปอย่างไรจากสระบุรี

### instruction
1. identify user group that has posted on social media
2. as a psychologist, please analyze did the users have posted those massageas 
3. classify sentiment of the post whether positive, neutal or negative. If negative following step 4, 5 and 6 otherwise go to step 5 and 6.
4. identify a concern or an issue
5. identify a wish of the users
6. return output in json format that contain

### example province
{example_province}

### example social media posts
{example_social_posts}

### example output format
{example_output}

### province
{province}

### social media posts
{social_posts}

### output format
""")

In [3]:
class IndicatorOutputParser(BaseOutputParser):
    """Parse the output of an LLM call to a comma-separated list."""


    def parse(self, social_analysis: dict):
        obj_social_analysis=json.loads(social_analysis)
        targets=[i.split(' ')[0][4:] for i in obj_social_analysis['relevance_sdg_target']]
        df2=pd.read_csv('sdg.csv')
        df2=df2.query("target == @targets")
        text_indicators=[]
        for idx,i in df2[['indicator','indicator_description']].iterrows():
            text_indicators.append({'indicator':i['indicator'], 'indicator_description':i['indicator_description']}  )
        obj_social_analysis['relevance_sdg_indicators']=text_indicators
        return obj_social_analysis

In [4]:
chain1 = prompt1 | model |  StrOutputParser() | IndicatorOutputParser()

In [5]:
prompt2 = ChatPromptTemplate.from_template("""
Form social post anylysis and the given sdg indicators, please anaylize challenges 
then select indicator and provide posible solutions 
that can fulfil the gap between user wishes and the indicator in the style of Bjorn Lomborg. 
return output only in json format

### example social post anylysis
{example_chain2_social_analysis}

### example output json format
{example_chain2_output}

### social post anylysis
{obj_social_analysis}

### output json format
""")

In [6]:
from langchain.output_parsers import PydanticOutputParser
from langchain.prompts import PromptTemplate
from langchain.pydantic_v1 import BaseModel, Field, validator
from typing import List
from langchain.output_parsers import OutputFixingParser

class Challenge(BaseModel):
    challenge_description: str = Field(description="a gap between the people wishes and the current situation")
    challenge_topic: str = Field(description="short name of the challenge")
    indicators: List[str] = Field(description="list of relevance SDG indicators")
    solutions: List[str] = Field(description="list of solutions for the challenges")


actor_query = "Extract information from the social post analysis"

parser = PydanticOutputParser(pydantic_object=Challenge)

prompt = PromptTemplate(
    template="Answer the query.\n{format_instructions}\n{query}\n",
    input_variables=["query"],
    partial_variables={"format_instructions": parser.get_format_instructions()},
)

new_parser = OutputFixingParser.from_llm(parser=parser, llm=ChatOpenAI())

chain2 = (
    {
        "obj_social_analysis": chain1, 
        "example_chain2_social_analysis": itemgetter("example_chain2_social_analysis"),
        "example_chain2_output": itemgetter("example_chain2_output")
    }
    | prompt2
    | model
    | StrOutputParser()
    |new_parser
)

In [7]:
def get_challenge_solutions(location="ประเทศไทย",posts=["นักเรียนจบแล้วมีทักษะไม่พร้อมสำหรับการทำงาน",]):
    with open('template.json', 'r') as openfile:
        template = json.load(openfile)
    template['province']=location
    template['social_posts']=posts
    
    c2=chain2.invoke(template)
    d={}
    d['challenge_topic']=c2.challenge_topic
    d['challenge_description']=c2.challenge_description
    d['indicators']=c2.indicators
    d['solutions']=c2.solutions
    return d
get_challenge_solutions()

{'challenge_topic': 'Job Readiness and Skills Development',
 'challenge_description': 'Recent graduates expressing concerns about their readiness for the workforce, indicating a need for improved job training and preparation programs.',
 'indicators': ["SDG_Indicator_4.4.1: Proportion of youth and adults with information and communications technology (ICT) skills by type of skill aligns with the users' wish for improved job training and relevant skills for employment."],
 'solutions': ['Enhance vocational and technical training programs to equip graduates with the necessary skills for employment, focusing on ICT skills and other relevant technical abilities demanded by the job market.',
  'Establish partnerships between educational institutions and industries to provide practical training and internships, bridging the gap between theoretical knowledge and practical workplace skills.',
  'Introduce career counseling and mentorship programs to guide graduates in identifying their strengt

In [9]:
df=pd.read_csv('posts.csv')
df.info()
df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 15195 entries, 0 to 15194
Data columns (total 7 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   index        15195 non-null  int64  
 1   Topic        15195 non-null  int64  
 2   Name         15195 non-null  object 
 3   Document     15195 non-null  object 
 4   Probability  15195 non-null  float64
 5   content      15195 non-null  object 
 6   province     15195 non-null  object 
dtypes: float64(1), int64(2), object(4)
memory usage: 831.1+ KB


,index,Topic,Name,Document,Probability,content,province
0,1,0,0_#รายงานฝนฟ้าอากาศ_ทางไหนฝนตกกันบ้างข้าว_กำลั...,ฝนตก … นอกหน้าต่าง #ฤดูฝน #วันฝนพรำ #ฝนตก,1.000000,ฝนตก … นอกหน้าต่าง #ฤดูฝน #วันฝนพรำ #ฝนตก,พิจิตร
1,2,0,0_#รายงานฝนฟ้าอากาศ_ทางไหนฝนตกกันบ้างข้าว_กำลั...,ฝนกำลังมาจาก อ.วังทอง ถ้าไม่แวะไหนก่อน ก็กำลัง...,0.950048,ฝนกำลังมาจาก อ.วังทอง ถ้าไม่แวะไหนก่อน ก็กำลัง...,พิษณุโลก
2,3,0,0_#รายงานฝนฟ้าอากาศ_ทางไหนฝนตกกันบ้างข้าว_กำลั...,ในตัวเมืองฝนตกนะครับ,1.000000,ในตัวเมืองฝนตกนะครับ,พิษณุโลก
3,4,0,0_#รายงานฝนฟ้าอากาศ_ทางไหนฝนตกกันบ้างข้าว_กำลั...,สวัสดีวันพฤหัสบดี 12 ตค 2566 05.00 น. \n🐓🐓🐓☕😁\...,1.000000,สวัสดีวันพฤหัสบดี 12 ตค 2566 05.00 น. \n🐓🐓🐓☕😁\...,ฉะเชิงเทรา
4,5,0,0_#รายงานฝนฟ้าอากาศ_ทางไหนฝนตกกันบ้างข้าว_กำลั...,ไหนบอกเข้าหนาว\nฝนโปรยแล้วในเมือง,1.000000,ไหนบอกเข้าหนาว\nฝนโปรยแล้วในเมือง,พิษณุโลก


In [13]:
df2=df[['content','province']]

In [37]:
df3=df2.groupby(['province','content']).agg({'province':['count']})

In [39]:
df3.reset_index()

,province,content,province
,,,count
0,global,"""โลตัส"" พลิกโฉมธุรกิจแบบ Transformational Grow...",1
1,global,#สีดาโคราช,2
2,global,.\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\t\...,5
3,global,1.ควรไปจังหวัดไหนก่อน\n2.มีเวลา 7 วันครับ\n\nร...,3
4,global,19 ต.ค. 2565 - 18:25 น.\n\nน้ำตาท่วมวัด! เผา...,1
...,...,...,...
9801,แม่ฮ่องสอน,📣ประกาศเขตพื้นที่❗️ห้ามดื่ม เครื่องดื่มแอลกอฮอ...,1
9802,แม่ฮ่องสอน,"🙏🏼พลังแห่งศรัทธาร่วมบูรณะ\n""สะพานซูตองเป้"" สะพ...",1
9803,แม่ฮ่องสอน,🚨สภ.ขุนยวม\nอำนวยการโดย\nพ.ต.อ.พิษณุ สมนึก\n ...,1


In [46]:
df3.to_excel('posts2.xlsx',engine='xlsxwriter')

/home/wasit/miniconda3/envs/dsmde/lib/python3.10/site-packages/xlsxwriter/worksheet.py:1252: UserWarning: Ignoring URL 'https://www.thaipbs.or.th/news/content/321541

จากกรณีที่ไทยพีบีเอสได้รายงานข่าวในข่าวภาคค่ำและในเว็บไซต์ข่าวไทยพีบีเอส เมื่อวันที่ 14 พ.ย.ที่ผ่านมา เรื่องเมนูที่จะใช้ในงานเลี้ยงผู้นำเขตเศรษฐกิจภูมิภาคเอเชีย-แปซิฟิก (เอเปค) โดยอ้างคำสัมภาษณ์ของผู้ประกอบการในพื้นที่ อ.ตากใบ จ.นราธิวาส ว่า ปลากุเลาเค็มตากใบที่จะใช้ในงานเลี้ยงไม่ใช่ปลากุเลาจากตากใบ ต่อมา น.ส.รัชดา ธนาดิเรก รองโฆษกรัฐบาล ชี้แจงว่า ปลากุเลาที่นำมาใช้ปรุงอาหารได้สั่งซื้อมาจากร้านป้าอ้วน ซึ่งเป็นปลากุเลาเค็มตากใบจริง
ไทยพีบีเอสขอชี้แจง ดังต่อไปนี้

1. ผู้สื่อข่าวลงพื้นที่ อ.ตากใบ ด้วยความตั้งใจที่จะนำเสนอความภาคภูมิใจของชาวตากใบที่ผลิตภัณฑ์ในพื้นที่ได้รับการยอมรับในระดับสากล แต่ได้รับข้อมูลจากผู้ประกอบการหลายร้านว่า ไม่มีการสั่งซื้อปลากุเลาจากร้านในพื้นที่ รวมถึงในไลน์กลุ่มวิสาหกิจชุมชนปลากุเลาตากใบ ก็มีการสอบถามกันว่ารัฐบาลซื้อปลาจากร้านไหน แต่ไม่มีใครยืนยันว่าได้รับการสั่งซื้อ

ก่อนหน้านี้รัฐบาลประชาสัมพัน

In [45]:
pip install xlsxwriter

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.8/154.8 kB 7.4 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.
